In [1]:

import numpy as np 
import matplotlib.pyplot as plt 
import matplotlib.ticker as mticker 
import os 
import itertools 
import mesa_reader as mr 


import utils.load_data as load_data
import utils.plotting.profile_plotting as profile_plotting 
import utils.plotting.history_plotting as history_plotting 
import utils.plotting.HR_diagram_plotting as HR_diagram_plotting
import utils.config.ui_options as ui_options 
import utils.config.plot_options as plot_options
import utils.config.stellar_evolution_data as stellar_evolution_data 
import utils.config.physical_constants as physical_constants 


# Automatically reload modules 
%load_ext autoreload 
%autoreload 2 

# Make matplotlib plots open in a separate interactive window 
%matplotlib qt 



In [22]:
# Testing cell 

history = load_data.load_history(stellar_evolution_data.data_folder/"M=1.0")
profile = load_data.load_profile(stellar_evolution_data.data_folder/"M=1.0", modelnum=300, history=history) 

# _ = profile_plotting.ProfilePlot.temp(profile, history=history) 
_ = profile_plotting.ProfilePlot.degeneracy(profile, history=history) 
# _ = profile_plotting.ProfilePlot.fusion(profile, history=history) 
# _ = profile_plotting.ProfilePlot.mu(profile, history=history) 
# _ = profile_plotting.ProfilePlot.composition(profile, history=history) 




In [196]:
# Calculate moment of inertia and M*R**2 

m = 10.0 
history = load_data.load_history(stellar_evolution_data.data_folder/f"M={m}")


# Calculate moment of inertia  
def calc_inertia(profile):

    # Sort arrays so they start at center of star, and convert to CGS units 
    ind_sort = np.argsort(profile.mass)
    mass_coord_sorted_g = profile.mass[ind_sort] * physical_constants.M_sun 
    radius_coord_sorted_cm = profile.radius[ind_sort] * physical_constants.R_sun
    
    # I = integral of 2/3 * r**2 over the mass coordinate (2/3 factor because its the I of a spherical shell)
    inertia = np.trapz(2/3* radius_coord_sorted_cm**2, x=mass_coord_sorted_g)
    
    return inertia 


ages = [] 
inertias = [] 
m_rsquareds = [] 
for modelnum in history.model_numbers_available: 
    print(f"{modelnum} / {np.max(history.model_numbers_available)}")
    profile = load_data.load_profile(stellar_evolution_data.data_folder/f"M={m}", modelnum=modelnum, history=history) 
    ages.append(history.star_age[modelnum-1]) 
    inertias.append(calc_inertia(profile))
    m_rsquareds.append(history.star_mass[modelnum-1]*physical_constants.M_sun * ((10**history.log_R[modelnum-1])*physical_constants.R_sun)**2)




# Plot moment of inertia and MR^2 
plt.figure(figsize=(15, 8))
plt.plot(ages, m_rsquareds, label="MR^2", lw=3) 
plt.plot(ages, inertias, label="Moment of inertia", lw=3) 
plt.yscale("log")
plt.xlabel("Age (years)")
plt.ylabel("grams * cm^2") 
title = f"Moment of Inertia vs time of a {m} M_sun star"
plt.title(title)
plt.grid(alpha=0.5) 
plt.legend() 
plt.savefig(title + ".jpg")


# Plot ratio of I to MR^2 to get the f-constant in I=fMR^2 
plt.figure(figsize=(15, 8))
plt.plot(ages, np.array(inertias)/np.array(m_rsquareds), lw=3) 
plt.xlabel("Age (years)")  
title = f"f vs time for a {m} M_sun star"
plt.title(title)
plt.grid(alpha=0.5) 
plt.savefig(title + ".jpg")








1 / 2550
50 / 2550
100 / 2550
150 / 2550
200 / 2550
250 / 2550
272 / 2550
300 / 2550
322 / 2550
350 / 2550
354 / 2550
400 / 2550
412 / 2550
419 / 2550
450 / 2550
494 / 2550
500 / 2550
550 / 2550
600 / 2550
650 / 2550
700 / 2550
750 / 2550
800 / 2550
850 / 2550
900 / 2550
950 / 2550
1000 / 2550
1050 / 2550
1100 / 2550
1150 / 2550
1200 / 2550
1250 / 2550
1300 / 2550
1350 / 2550
1400 / 2550
1450 / 2550
1500 / 2550
1550 / 2550
1600 / 2550
1650 / 2550
1700 / 2550
1750 / 2550
1800 / 2550
1850 / 2550
1900 / 2550
1950 / 2550
2000 / 2550
2050 / 2550
2100 / 2550
2150 / 2550
2200 / 2550
2250 / 2550
2300 / 2550
2350 / 2550
2400 / 2550
2450 / 2550
2500 / 2550
2550 / 2550


In [ ]:
# Calculate moment of inertia and M*R**2 

m = 10.0 
history = load_data.load_history(stellar_evolution_data.data_folder/f"M={m}")


# Calculate internal energy 
def calc_internal_energy(profile):

    # Sort arrays so they start at center of star, and convert to CGS units 
    ind_sort = np.argsort(profile.mass) 
    mass_coord_sorted_g = profile.mass[ind_sort] * physical_constants.M_sun 
    temp_sorted = 10**profile.logT[ind_sort] 
    mu_sorted = profile.mu[ind_sort]

    # E = 3/2 * kT / (mu*m_p) 
    energy = np.trapz(3/2 * physical_constants.k * temp_sorted / (mu_sorted*physical_constants.m_p), x=mass_coord_sorted_g)
    return energy 



ages = [] 
energies = [] 
g_msquared_over_r = [] 
for modelnum in history.model_numbers_available: 
    print(f"{modelnum} / {np.max(history.model_numbers_available)}")
    profile = load_data.load_profile(stellar_evolution_data.data_folder/f"M={m}", modelnum=modelnum, history=history) 
    ages.append(history.star_age[modelnum-1]) 
    energies.append(calc_internal_energy(profile))
    g_msquared_over_r.append(physical_constants.G * (history.star_mass[modelnum-1]*physical_constants.M_sun)**2 / ((10**history.log_R[modelnum-1])*physical_constants.R_sun))




# Plot GM^2 / R 
plt.figure(figsize=(15, 8))
plt.plot(ages, g_msquared_over_r, label="GM^2 / R", lw=3) 
plt.plot(ages, energies, label="3/2*kT/(mu*m_p)", lw=3)
plt.yscale("log")
plt.xlabel("Age (years)")
plt.ylabel("ergs") 
title = f"Gravitational and Thermal Energy of a {m} M_sun star"
plt.title(title)
plt.grid(alpha=0.5) 
plt.legend() 
plt.savefig(title + ".jpg")



# Plot ratio 
plt.figure(figsize=(15, 8))
plt.plot(ages, np.array(energies)/np.array(g_msquared_over_r), lw=3) 
plt.ylim((0.5, 1.4))
plt.xlabel("Age (years)") 
title = f"Ratio of Thermal Energy to Gravitational Energy of a {m} M_sun star"
plt.title(title)
plt.grid(alpha=0.5) 
plt.savefig(title + ".jpg")







1 / 2550
50 / 2550
100 / 2550
150 / 2550
200 / 2550
250 / 2550
272 / 2550
300 / 2550
322 / 2550
350 / 2550
354 / 2550
400 / 2550
412 / 2550
419 / 2550
450 / 2550
494 / 2550
500 / 2550
550 / 2550
600 / 2550
650 / 2550
700 / 2550
750 / 2550
800 / 2550
850 / 2550
900 / 2550
950 / 2550
1000 / 2550
1050 / 2550
1100 / 2550
1150 / 2550
1200 / 2550
1250 / 2550
1300 / 2550
1350 / 2550
1400 / 2550
1450 / 2550
1500 / 2550
1550 / 2550
1600 / 2550
1650 / 2550
1700 / 2550
1750 / 2550
1800 / 2550
1850 / 2550
1900 / 2550
1950 / 2550
2000 / 2550
2050 / 2550
2100 / 2550
2150 / 2550
2200 / 2550
2250 / 2550
2300 / 2550
2350 / 2550
2400 / 2550
2450 / 2550
2500 / 2550
2550 / 2550


In [265]:
import matplotlib as mpl 

for n in range(12): 
    plt.plot([0+n,1+n], [0,1], color=mpl.colormaps["Set3"](n/12+1/24), lw=50)


